In [2]:
from gqlalchemy import Memgraph

# Connect to Memgraph
db = Memgraph(host='127.0.0.1', port=7687)

# Query 1: Find all rooms
print("1. All Rooms in the Lair:")
rooms = db.execute_and_fetch("MATCH (r:Room) RETURN r.name as name, r.danger_level as danger")
for room in rooms:
    print(f"   - {room['name']}")


1. All Rooms in the Lair:


In [ ]:
from langchain_core.tools import Tool

def make_find_exit_passageways(agent):
    def run(x, **kwargs):
        query = """
        MATCH (r:Room {name: $room_name})-[c:CONNECTS_TO]->(connected:Room)
        RETURN connected.name as room, c.passageway as passageway
        """
        exits = db.execute_and_fetch(query, {'room_name': agent.current_room})
        if exits:
            return f"You see exits: {[e['passageway'] for e in exits]}"
        return "No exits here."
    
    return Tool(name="find_exit_passageways", func=run, description="Find exits. No arguements.", args_schema=None)


def make_use_passageway(agent):
    def run(passageway: str):
        query = """
        MATCH (r:Room {name: $current_room})-[c:CONNECTS_TO]->(connected:Room)
        WHERE c.passageway = $passageway
        RETURN connected.name as room, connected.description as description
        """
        result = list(db.execute_and_fetch(query, {
            "current_room": agent.current_room,
            "passageway": passageway
        }))
        
        if result:
            agent.current_room = result[0]["room"]
            return f"Moved to {agent.current_room}: {result[0]['description']}"
        
        return f"No passageway '{passageway}' from '{agent.current_room}'."
    
    return Tool(name="use_passageway", func=run, description="Use a passageway.")


In [ ]:
from abc import ABC

import dotenv
from langchain.agents import create_agent

dotenv.load_dotenv()

class PartyMember(ABC):
    def __init__(self):
        super().__init__()

class LeaderAgent(PartyMember):
    def __init__(self, conversation_id):
        self.conversation_id = conversation_id
        self.current_room = None
        self.messages = []

        self.find_exit_tool = make_find_exit_passageways(self)
        self.use_passageway_tool = make_use_passageway(self)

        self.agent = create_agent(
            model="o3-mini",
            tools=[self.find_exit_tool, self.use_passageway_tool],
            system_prompt=leader_system_prompt,
        )

    def journey(self, starting_room):
        """
        """

        self.current_room = starting_room
        # append the user's message to the chat history
        self.messages.append({"role": "user", "content": "You enter the dungeon."})

        print(self.messages)
        try:
            messages_with_response = self.agent.invoke(
                input={
                    "messages": [
                        {"role": m["role"], "content": m["content"]}
                        for m in self.messages
                    ]
                },
                config={
                    "metadata": {"conversation_id": self.conversation_id},
                    "tags": [f"conversation:{self.conversation_id}"],
                },
            )
        except Exception as e:
            raise e

        # extract the last message from the bot in the chain
        response = messages_with_response["messages"][-1]

        # add the message from the LLM to the chat history
        self.messages.append(
            {
                "role": "assistant",
                "content": response.content,
                "tokens_used": response.usage_metadata["total_tokens"],
            }
        )

        return response.content

In [ ]:
import uuid

leader = LeaderAgent( uuid.uuid4())

leader.journey("Collapsed Entrance Tunnel")

[{'role': 'user', 'content': 'You enter the dungeon.'}]


'I have now thoroughly explored the dungeon. From the dungeon entrance, I journeyed through the Hall of Echoing Anvils, down the Broken hallway to the Fallen Bridge Over Chasm, across the narrow ledge to the Abandoned Dwarven Market, then into the shimmering Crystal Lantern Grove. From there I ventured into both the Runic tunnel leading to the sealed ritual vault and the Grand stone stairway to the ruined Throne of the Deep King. I even traced back a route through the Collapsed Entrance Tunnel. Each passage has been examined and has led back into the network with no new unexplored exits remaining.\n\nAt this point, all discovered corridors seem to have been traversed. Would you like to backtrack further, or exit the dungeon?'